# Clinic Data Visualization

### Alex Lu | Spring Q 17-18 

First, let's import all of the stuff that we'll need

In [21]:
import numpy
import pandas as pd
import matplotlib.pyplot as plt
import csv
import seaborn as sns
from utils.file_ops import *
%matplotlib inline

Now, we can start loading the data from our ad hoc report pulls. The first batch of data that we have contains: 
* Age(Year) 
* AppointmentID 
* AppointmentLocation
* City
* Comment
* DataFirstKnown
* DateTime
* DepartmentName
* EstablishedDateTime
* FreeTextDescription
* LocationAbbreviation
* MRN
* PatientID
* ProblemComment
* Reason
* VisitType
* Zip

In [22]:
path = "data/big_clinic_pull_5.13.csv"

df_big_pull = returnDF(path)
df_big_pull.head()


preparing to open csv at: data/big_clinic_pull_5.13.csv


Error: new-line character seen in unquoted field - do you need to open the file in universal-newline mode?

While we haven't decided exactly what we want to pull from this yet, we can confirm that the comment field seems to be very blank. The MRN field also seems to be blank. Further, the paradigm of pulling by appointment ID seems less effective than pulling by visitID. In fact, until we pivot by patientID or appointmentID, we're even going to have a tough time distributing patients by Zip because we have some 1:n joins in here. 

In [16]:

path2 = "data/VisitToDiagnosis5.14.csv"

df_V2D = pd.read_csv(path2, error_bad_lines=False, header=0)
df_V2D.head()



,VisitId,LocationAbbreviation,PatientId,MRN,Age(Year),VisitType,DateFirstKnown,VisitDateTime,Diagnosis,ICDCode,Comment
0,18529,AFC,6526,NaN,77,ARBOR GENERAL VISIT,4/29/2018 9:10:07 AM,4/29/2018 10:00:00 AM,BENIGN POSITIONAL VERTIGO,386.11,NaN
1,18579,AFC,6526,NaN,77,LAB FOLLOW-UP NOTE,4/29/2018 9:10:07 AM,5/6/2018 12:26:24 PM,NaN,NaN,NaN
2,18534,AFC,6528,NaN,73,ARBOR GENERAL VISIT,4/29/2018 10:28:15 AM,4/29/2018 11:15:00 AM,"DIABETES MELLITUS, TYPE II",250.00,NaN
3,18576,AFC,6528,NaN,73,LAB FOLLOW-UP NOTE,4/29/2018 10:28:15 AM,5/6/2018 11:57:41 AM,NaN,NaN,NaN
4,18565,AFC,6542,NaN,72,ARBOR GENERAL VISIT,5/6/2018 8:56:51 AM,5/6/2018 9:15:00 AM,"ANGINA PECTORIS, UNSTABLE",411.1,NaN


In [17]:
path3 = 'data/QIdudes.csv'
df_QI = returnDF(path3)


preparing to open csv at: data/QIdudes.csv
done preparing dataframe


We want to filter on the time of visit for longitudinal visualization. First, we should do a conversion on the format of the time string to make it more workable.

In [20]:
df_QI['ts'] = pd.to_datetime((df_QI['VisitDate']))
df_QI.head(30)


,LastModifiedDateTime,LocationName,Note PropertyName,Note PropertyValue,PatientNumber,ProviderName,VisitDate,ts
0,5/13/2018 2:29:32 PM,ARBOR,QIDiabetes,No,N0066993,"JAFFE, GILAD",5/13/2018 1:45:00 PM,2018-05-13 13:45:00
1,5/13/2018 2:29:32 PM,ARBOR,QIPap,Not Indicated,N0066993,"JAFFE, GILAD",5/13/2018 1:45:00 PM,2018-05-13 13:45:00
2,5/13/2018 2:29:32 PM,ARBOR,QIPHQ1,No,N0066993,"JAFFE, GILAD",5/13/2018 1:45:00 PM,2018-05-13 13:45:00
3,5/13/2018 2:29:32 PM,ARBOR,StudentAssessment,Upon examination pt has significant pain in th...,N0066993,"JAFFE, GILAD",5/13/2018 1:45:00 PM,2018-05-13 13:45:00
4,5/13/2018 2:29:32 PM,ARBOR,WriteStudentAssessment,Yes,N0066993,"JAFFE, GILAD",5/13/2018 1:45:00 PM,2018-05-13 13:45:00
5,5/13/2018 2:29:32 PM,ARBOR,QIPHQ2,No,N0066993,"JAFFE, GILAD",5/13/2018 1:45:00 PM,2018-05-13 13:45:00
6,5/13/2018 1:35:13 PM,ARBOR,QIPHQ2Reason1,Pt has been waiting for 2 hours and would like...,N0066986,"CHARON, MINA",5/13/2018 10:28:00 AM,2018-05-13 10:28:00
7,5/13/2018 1:35:13 PM,ARBOR,QIPHQ2,Not Questioned,N0066986,"CHARON, MINA",5/13/2018 10:28:00 AM,2018-05-13 10:28:00
8,5/13/2018 1:35:13 PM,ARBOR,QIPHQ2Reason2,as above,N0066986,"CHARON, MINA",5/13/2018 10:28:00 AM,2018-05-13 10:28:00
9,5/13/2018 1:35:13 PM,ARBOR,QIDiabetes,No,N0066986,"CHARON, MINA",5/13/2018 10:28:00 AM,2018-05-13 10:28:00


In [19]:
arbor_data = df_QI[df_QI['LocationName'] == 'ARBOR']
pfc_data = df_QI[df_QI['LocationName'] == 'PFC']

arbor_pivot = arbor_data.pivot_table(index='PatientNumber',columns='Note PropertyName',values='Note PropertyValue', aggfunc='first')
pfc_pivot = pfc_data.pivot_table(index='PatientNumber',columns='Note PropertyName',values='Note PropertyValue', aggfunc='first')

#df_QI.head(5)
arbor_pivot.head(10)

complete_qisummary(arbor_pivot)

#pfc_pivot.head(5)

NameError: name 'complete_qisummary' is not defined